In [ ]:
import torch
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader
from torch import nn


rain_type_df = pd.read_csv("../processed/rain_type.csv", parse_dates=["Date Time"], index_col="Date Time")
rain_type_df = rain_type_df.drop(columns=['Rain_Rate (mm/h)'])
rain_type_df = rain_type_df.drop(columns=['rain (mm)'])
rain_type_df = rain_type_df.drop(columns=['raining (s)'])

X = rain_type_df.drop(columns=[
    "Rain_Type_Cloudburst", "Rain_Type_Heavy_Rain", "Rain_Type_Moderate_Rain",
    "Rain_Type_No_Rain", "Rain_Type_Shower", "Rain_Type_Very_Heavy_Rain", "Rain_Type_Weak_Rain"
]).values

y = rain_type_df[[
    "Rain_Type_Cloudburst", "Rain_Type_Heavy_Rain", "Rain_Type_Moderate_Rain",
    "Rain_Type_No_Rain", "Rain_Type_Shower", "Rain_Type_Very_Heavy_Rain", "Rain_Type_Weak_Rain"
]].values

# new_X = new_X.astype(np.float32)
X = X.astype(np.float32)
y = y.astype(np.float32)

# Train test split
training_data_len = math.ceil(len(rain_type_df) * .9)

# Splitting the dataset
train_data = rain_type_df[:training_data_len].iloc[:]
test_data = rain_type_df[training_data_len:].iloc[:]

input_hours = 28
output_hours = 12

X_train, y_train = [], []
timestamps = train_data.index

for i in range(len(train_data) - input_hours - output_hours):
    # Extract input
    X_train.append(train_data.iloc[i:i+input_hours, :19])

    # Extract output
    y_train.append(train_data.iloc[i+input_hours:i+input_hours+output_hours, 19:])

# Convert to torch tensor arrays
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

print("Input shape:", X_train.shape)  
print("Output shape:", y_train.shape) 


X_test, y_test = [], []
timestamps = test_data.index

for i in range(len(test_data) - input_hours - output_hours):
    # Extract input
    X_test.append(test_data.iloc[i:i+input_hours, :19])

    # Extract output
    y_test.append(test_data.iloc[i+input_hours:i+input_hours+output_hours, 19:])

# Convert to torch tensor arrays
X_test, y_test = np.array(X_test), np.array(y_test)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

print("Input shape:", X_test.shape)  
print("Output shape:", y_test.shape) 

input_size = X.shape[1]
output_size = 7

class WeatherDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

class LSTMConfig1(nn.Module):
    def __init__(self, input_size=input_size, output_size=output_size, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 128, batch_first=True)
        self.lstm2 = nn.LSTM(128, 512, batch_first=True)
        self.lstm3 = nn.LSTM(512, 512, batch_first=True)
        self.lstm4 = nn.LSTM(512, 256, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(256, output_size * output_hours)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out)
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        out, _ = self.lstm4(out)
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, output_hours, output_size)
        return out
    

class LSTMConfig2(nn.Module):
    def __init__(self, input_size=input_size, output_size=output_size, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 256, batch_first=True)
        self.lstm2 = nn.LSTM(256, 2048, batch_first=True, dropout=dropout)
        self.lstm3 = nn.LSTM(2048, 2048, batch_first=True, dropout=dropout)
        self.lstm4 = nn.LSTM(2048, 1024, batch_first=True, dropout=dropout)
        self.lstm5 = nn.LSTM(1024, 256, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(256, output_size * output_hours)

    def forward(self, x):
        out, _ = self.lstm(x)
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        out, _ = self.lstm4(out)
        out = self.dropout(out)
        out, _ = self.lstm5(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, output_hours, output_size)
        return out


class LSTMConfig3(nn.Module):
    def __init__(self, input_size=input_size, output_size=output_size, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 256, batch_first=True)
        self.lstm2 = nn.LSTM(256, 512, batch_first=True, dropout=dropout)
        self.lstm3 = nn.LSTM(512, 1024, batch_first=True, dropout=dropout)
        self.lstm4 = nn.LSTM(1024, 512, batch_first=True, dropout=dropout)
        self.lstm5 = nn.LSTM(512, 256, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(256, output_size * output_hours)

    def forward(self, x):
        out, _ = self.lstm(x)
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        out, _ = self.lstm4(out)
        out = self.dropout(out)
        out, _ = self.lstm5(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, output_hours, output_size)
        return out


class LSTMConfig4(nn.Module):
    def __init__(self, input_size=input_size, output_size=output_size, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 256, batch_first=True)
        self.lstm2 = nn.LSTM(256, 1024, batch_first=True, dropout=dropout)
        self.lstm3 = nn.LSTM(1024, 1024, batch_first=True, dropout=dropout)
        self.lstm4 = nn.LSTM(1024, 512, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(512, output_size * output_hours)

    def forward(self, x):
        out, _ = self.lstm(x)
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        out, _ = self.lstm4(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, output_hours, output_size)
        return out


class LSTMConfig5(nn.Module):
    def __init__(self, input_size=input_size, output_size=output_size, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 64, batch_first=True)
        self.lstm2 = nn.LSTM(64, 256, batch_first=True, dropout=dropout)
        self.lstm3 = nn.LSTM(256, 512, batch_first=True, dropout=dropout)
        self.lstm4 = nn.LSTM(512, 128, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(128, output_size * output_hours)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.dropout(out)
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        out, _ = self.lstm3(out)
        out = self.dropout(out)
        out, _ = self.lstm4(out)
        out = self.dropout(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, output_hours, output_size)
        return out


class LSTMConfig6(nn.Module):
    def __init__(self, input_size=input_size, output_size=output_size, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 128, batch_first=True)
        self.lstm2 = nn.LSTM(128, 512, batch_first=True, dropout=dropout)
        self.lstm3 = nn.LSTM(512, 256, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(256, output_size * output_hours)

    def forward(self, x):
        out, _ = self.lstm(x)
        out, _ = self.lstm2(out)
        out = self.dropout(out)
        out, _ = self.lstm3(out)
        out = self.fc(out[:, -1, :])
        out = out.view(-1, output_hours, output_size)
        return out

    
class LSTMConfig7(nn.Module):
    def __init__(self, input_size=input_size, output_size=output_size, dropout=0.0):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 128, batch_first=True)
        self.lstm2 = nn.LSTM(128, 512, batch_first=True)
        # self.lstm3 = nn.LSTM(512, 256, batch_first=True)
        self.fc = nn.Linear(512, output_size * output_hours)
        # self.fc = nn.Linear(256, output_hours)

    def forward(self, x):
        out, _ = self.lstm(x)
        out, _ = self.lstm2(out)
        out = self.fc(out[:, -1, :]) 
        out = out.view(-1, output_hours, output_size)
        return out
    
class LSTMConfig8(nn.Module):
    def __init__(self, input_size=input_size, output_size=output_size, dropout=0.3):
        super().__init__()
        self.lstm1 = nn.LSTM(input_size, 256, batch_first=True)
        self.dropout1 = nn.Dropout(dropout)
        
        self.lstm2 = nn.LSTM(256, 128, batch_first=True)
        self.dropout2 = nn.Dropout(dropout)
        
        self.fc = nn.Linear(128, output_size * output_hours)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.dropout1(out)

        out, _ = self.lstm2(out)
        out = self.dropout2(out)

        out = self.fc(out[:, -1, :])  # Take only the last time step
        out = out.view(-1, output_hours, output_size)
        return out


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)



train_dataset = WeatherDataset(X_train, y_train)
test_dataset = WeatherDataset(X_test, y_test)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

models = [LSTMConfig1(), LSTMConfig2(), LSTMConfig3(), LSTMConfig4(), LSTMConfig5(), LSTMConfig6()]

learning_rate = 0.001
num_epochs = 100

for model in models:
    model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        accuracy_hist_train = 0

        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            is_correct = (torch.argmax(pred, dim=2) == torch.argmax(y_batch, dim=2))
            accuracy_hist_train += is_correct.sum()

        accuracy_hist_train = accuracy_hist_train.float() / (len(train_loader.dataset) * output_hours)
        print(f'Epoch {epoch} Accuracy: {accuracy_hist_train}')


    model.eval()
    accuracy_hist_test = 0

    with torch.no_grad():
        for x_test, y_test in test_loader:
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            pred = model(x_test)

            is_correct = (torch.argmax(pred, dim=2) == torch.argmax(y_test, dim=2))
            accuracy_hist_test += is_correct.sum()

        accuracy_hist_test = accuracy_hist_test.float() / (len(test_loader.dataset) * output_hours)
        print(f'{model.__class__.__name__} Accuracy: {accuracy_hist_test}')
    
    torch.save(model, f'{model.__class__.__name__}.pt')
    

Input shape: torch.Size([27712, 28, 19])
Output shape: torch.Size([27712, 12, 7])
Input shape: torch.Size([3043, 28, 19])
Output shape: torch.Size([3043, 12, 7])
cuda:0
torch.Size([16, 28, 19]) torch.Size([16, 12, 7])
Epoch 0 Accuracy: 0.13985517621040344
Epoch 1 Accuracy: 0.21639542281627655
Epoch 2 Accuracy: 0.22136017680168152
Epoch 3 Accuracy: 0.2405305653810501
Epoch 4 Accuracy: 0.2532055974006653
Epoch 5 Accuracy: 0.264945387840271
Epoch 6 Accuracy: 0.26568514108657837
Epoch 7 Accuracy: 0.2657933831214905
Epoch 8 Accuracy: 0.26243144273757935
Epoch 9 Accuracy: 0.26062414050102234
Epoch 10 Accuracy: 0.26544755697250366
Epoch 11 Accuracy: 0.2752477824687958
Epoch 12 Accuracy: 0.27192190289497375
Epoch 13 Accuracy: 0.272481232881546
Epoch 14 Accuracy: 0.2837038040161133
Epoch 15 Accuracy: 0.28132516145706177
Epoch 16 Accuracy: 0.2809913754463196
Epoch 17 Accuracy: 0.280570387840271
Epoch 18 Accuracy: 0.2800351083278656
Epoch 19 Accuracy: 0.2807357609272003
Epoch 20 Accuracy: 0.28314